In [1]:
import plotly as py
import plotly.graph_objs as go
import random as rand
import numpy as np
import math
import ipywidgets as widgets

py.offline.init_notebook_mode(connected=True)

In [2]:
minxrange, maxxrange, minyrange, maxyrange = 0,0,0,0

def fitnessP1(x,y):
    global minxrange, maxxrange, minyrange, maxyrange
    minxrange, maxxrange, minyrange, maxyrange = -2,2,-3,3
    a = (4-2.1*(x**2)+(x**4)/3)*(x**2)+x*y+(-4 + 4*(y**2))*(y**2)
    return a

def fitnessP2(x,y):
    global minxrange, maxxrange, minyrange, maxyrange
    minxrange, maxxrange, minyrange, maxyrange = -5,10,0,15
    a = (y-(5.1/(4*(math.pi)**2))*(x**2)+(5/math.pi)*x)**2+10*(1-1/(8*math.pi))*math.cos(x)+10
    return a

def fitnessP3(x,y):
    global minxrange, maxxrange, minyrange, maxyrange
    minxrange, maxxrange, minyrange, maxyrange = -100,100,-100,100
    a = 1+1/200*(x**2+y**2)-(math.cos(x/math.sqrt(1))*math.cos(y/math.sqrt(2)))
    return a

switcher = {
    0: fitnessP1,
    1: fitnessP2,
    2: fitnessP3
}

def fitness(funcIndex,x,y):
    func = switcher.get(funcIndex, lambda x,y: "Invalid index") # Get the function from switcher dictionary
    return func(x,y) # Execute the function

In [3]:
iterations = 1000
data = [[]]*5
data[0] = [10,30,1.1,[]] # parent, child, gama, len(parent)+1 size array of various parents results, array of arrays
data[1] = [10,30,2.5,[]]
data[2] = [10,20,1.1,[]]
data[3] = [5,30,2.5,[]]
data[4] = [15,30,2.5,[]]

In [4]:
results = [] # diffrent problems' diffret seed results
for r in range(len(switcher)):
    problemResult = [] # same problem diffrent seeds results
    for d in data: # iterate on seeds
        m = d[0] # parents
        l = d[1] # childs
        d[3] = [[] for _ in range(m+1)] # len(parent)+1 size array of various parents fitnesses, array of arrays
        parent_x = [0.0]*m
        parent_y = [0.0]*m
        child_x = [0.0]*l
        child_y = [0.0]*l
        z_fitness = [0.0]*m # parents fitness values
        t_fitness = [0.0]*l # children fitness values
        gama = d[2] # can be variable or constant - move function

        # family members production and reproduction
        for i in range(m):
            parent_x[i] = rand.uniform(minxrange,maxxrange)
            parent_y[i] = rand.uniform(minyrange,maxyrange)
            z_fitness[i] = fitness(r,parent_x[i],parent_y[i])

        p = 0 # index, or you can start from 1
        while p < iterations:
            k = 0
            s = 0.0
            for i in range(m):
                for j in range(int(l/m)):
                    moving_x = rand.gauss(0,1)*gama # normal random - !needs to be revised!
                    moving_y = rand.gauss(0,1)*gama # normal random - !needs to be revised! 
                    child_x[k] = parent_x[i] + moving_x
                    child_y[k] = parent_y[i] + moving_y
                    t_fitness[k] = fitness(r,child_x[k],child_y[k])
                    if t_fitness[k] < z_fitness[i]:
                        s+=1.0
                    k+=1
            # mutation
            if s / l > 0.2:
                gama = gama*(1/0.85)
            else:
                gama = gama * 0.85
            # m and l selection
            for i in range(m):
                for j in range(l):
                    if t_fitness[j] < z_fitness[i]:
                        parent_x[i] = child_x[j]                
                        parent_y[i] = child_y[j]
                        z_fitness[i] = t_fitness[j]
                        t_fitness[j] = 999999999999999 # eleminate using a very big number
                d[3][i].append(z_fitness[i])
            p+=1
            d[3][m].append(min(z_fitness))
        problemResult.append(d[3][m])
    results.append(problemResult)

In [5]:
x = np.linspace(0,iterations,iterations)
layout = go.Layout(
    title='Evaluation Strategies Algorithm Module',
    yaxis=dict(
        title='fitness'
    ),
    xaxis=dict(
        title='iteration'
    )
)

def update_plot(graphs,graphChildren):
    data = []
    for i in graphs:
        for j in graphChildren:
            trace1 = go.Scatter(
                x=x,
                y=results[i-1][j-1],
                mode='lines',
                name='prob{} seed{}'.format(i,j),
                line=dict(
                    shape='spline'
                )
            )
            data.append(trace1)
    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

graphs = widgets.SelectMultiple(options=list(range(1,len(results)+1)), value=(1, ),description='Problem Index')
graphChildren = widgets.SelectMultiple(options=list(range(1,len(results[0])+1)), value=(1, ),description='Seed Index')

widgets.interactive(update_plot,graphs=graphs, graphChildren=graphChildren)
#widgets.interactive(update_plot,graphs=graphs)


interactive(children=(SelectMultiple(description='Problem Index', index=(0,), options=(1, 2, 3), value=(1,)), SelectMultiple(description='Seed Index', index=(0,), options=(1, 2, 3, 4, 5), value=(1,)), Output()), _dom_classes=('widget-interact',))